In [ ]:
################################################################################
# MASTERARBEIT - SKRIPT 13:
# EXPERIMENT 3.5 (Forschungsfrage 3) - AUSREISSERERKENNUNG (LLM-Basis)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Forschungsfrage 3):
#
# 1. (Basis): Misst den Einfluss der LLM-Inkonsistenzbehebung (FF2.2) auf die 
#    Ausreißererkennung (Erweiterung auf 'price'/'saving').
#
# 2. (Datenbasis): Lädt den vom LLM reparierten Datensatz (2.2_rfd_repaired_llm.csv).
#
# 3. (Zielspalten/Kontext): Erweitert die Ausreißerbewertung auf 
#    die reparierten Spalten ('price', 'saving').
#
# 4. (Methodik): Wendet den semantischen LLM-Ansatz (FF1) auf der bereinigten Basis an.
#
# 5. (Detektion & Begründung): Das LLM wird angewiesen, die binäre Entscheidung 
#    und die Begründung (für Forschungsfrage 4) zurückzugeben.
#
# 6. (Speichern): Speichert die Ergebnisse (Indizes und Begründungen).
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import warnings
import json
import time
from anthropic import Anthropic, RateLimitError, APIError

# --- GLOBALE KONSTANTEN FÜR DIESES EXPERIMENT ---
# Wir replizieren FF1, aber die Basis ist jetzt die reparierte Datei.
DATEIPFAD_LLM_REPAIRED = 'ergebnisse/2.2_rfd_repaired_llm.csv'
DATEIPFAD_OUTPUT_INDIZES = 'ergebnisse/3.5_llm_ausreisser_indizes.csv'
DATEIPFAD_OUTPUT_KOMPLETT = 'ergebnisse/3.5_llm_komplette_ergebnisse.csv'

# ZIELSPALTEN: FF1-Spalten + die durch FF2 reparierten Spalten
zielspalten = ['replies', 'views', 'votes', 'price', 'saving']

# --- TECHNISCHE KONSTANTEN ---
MODELL_NAME = "claude-sonnet-4-5-20250929"
MAX_TOKENS = 500  # Weniger Tokens als bei Reparatur, da keine ganze Zeile zurückkommt

# Technische Warnungen ignorieren
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

print("Lade Anthropic-Bibliothek...")
try:
    print("Anthropic (Claude) Bibliothek erfolgreich geladen.")
except Exception:
    sys.exit("Skript gestoppt.")

print("Alle Bibliotheken sind bereit.")
print("=" * 70)

#
################################################################################

# --- SCHRITT 2: DATEN LADEN, API-CLIENT & DATENVORBEREITUNG ---
print("\n--- Schritt 2: Reparierte Daten laden & API-Client initialisieren ---")

# 1. Laden der LLM-reparierten Daten
try:
    # LLM-Daten laden (Index wird mitgeladen)
    df_llm = pd.read_csv(DATEIPFAD_LLM_REPAIRED, index_col='original_index')
    df_llm = df_llm.reset_index(drop=False)  # Index als Spalte 'original_index' beibehalten
    print(f"Datensatz geladen: {DATEIPFAD_LLM_REPAIRED} ({df_llm.shape[0]} Zeilen)")
except FileNotFoundError:
    print("FEHLER: Reparierte Basisdatei nicht gefunden.")
    sys.exit("Skript gestoppt.")

# 2. Typkonvertierung (ZWINGEND für die Analyse)
# Der Reparaturschritt von FF2 macht die Konvertierung möglich.
try:
    df_llm['price'] = pd.to_numeric(df_llm['price'], errors='coerce')
    df_llm['saving'] = pd.to_numeric(df_llm['saving'], errors='coerce')
    df_llm[['price', 'saving']] = df_llm[['price', 'saving']].fillna(0)  # NaN mit 0 füllen
    print("STATUS: Spalten 'price'/'saving' für LLM-Analyse vorbereitet (Float/NaN-bereinigt).")
except Exception:
    print("FEHLER: Konnte 'price'/'saving' nicht konvertieren. Daten sind nicht numerisch.")
    sys.exit("Skript gestoppt.")

# 3. Initialisierung des API-Clients
try:
    client = Anthropic()
    print(f"Anthropic (Claude) API-Client und Modell '{MODELL_NAME}' initialisiert.")
except Exception:
    print("FEHLER: Der Anthropic API-Client konnte nicht initialisiert werden.")
    sys.exit("Skript gestoppt.")

# 4. Datenvorbereitung (NaN zu Leerstring)
# Da das LLM-Reparaturskript (Skript 08) bereits alle numerischen NaNs in 0 umgewandelt hat,
# müssen wir hier nur die Metadaten-NaNs (Textspalten) zu Leerstrings konvertieren.
df_llm_input = df_llm.copy().fillna('')

print("=" * 70)

#
################################################################################

# SCHRITT 3: Definition der Prompt-Strategie und Hilfsfunktionen
print("\n--- Schritt 3: Prompt-Strategie und Hilfsfunktionen ---")

# --- 3.1: SYSTEM_PROMPT (FF1-Logik: Ausreißer suchen + Begründung) ---
SYSTEM_PROMPT = """
Du bist ein Modell zur kontextuellen Ausreißererkennung in bereinigten, tabellarischen Daten.
Deine Aufgabe ist es, einzelne Datenzeilen zu bewerten.

DATENSATZKONTEXT:
Die Daten stammen aus einem Online-Deal-Forum und wurden bereits bereinigt (Datentypen sind korrigiert).
Wichtige Spalten sind u.a.:
- 'title': Titel des Deals/Posts
- 'votes': Summe aus Up- und Downvotes
- 'replies': Anzahl der Antworten
- 'views': Anzahl der Aufrufe
- 'price': Angebotspreis (numerisch bereinigt)
- 'saving': Ersparnis (numerisch bereinigt)

ZIELSPALTEN (ZU BEWERTEN):
Bewerte ausschließlich die Werte in den Spalten:
['replies', 'views', 'votes', 'price', 'saving'].

KONTEXT:
Nutze alle anderen Spalten der Zeile (z.B. 'title', 'source', 'thread_category')
als Kontextsignale. Sie geben Hinweise auf Produktart, Marktwert, Beliebtheit und Deal-Charakter.

GRUNDIDEE:
Beurteile, ob die numerischen Werte in den Zielspalten im Verhältnis zum Kontext
ungewöhnlich (Ausreißer) oder erwartbar (Normal) sind.

- Ein Wert ist ein Ausreißer, wenn er für die Art des Produkts oder Deals im 'title'
  extrem untypisch erscheint (z.B. extrem hoher Preis für ein Billigprodukt,
  oder extrem hohe Views für ein Nischenthema).
- Beachte: Die Daten sind technisch bereinigt. Bewerte jetzt die inhaltliche
  Auffälligkeit der Zahlenwerte im Kontext.

ORIENTIERUNG:
- 'price' & 'saving': Ist der Preis für das im Titel genannte Produkt realistisch?
  (z.B. Laptop für 5€ -> Ausreißer? Oder Laptop für 5000€ -> Ausreißer?)
- 'engagement' ('views', 'replies', 'votes'): Passt die Aktivität zur Attraktivität des Deals?
- Negative 'votes' sind weiterhin möglich und kein alleiniger Grund für einen Ausreißer.

AUSGABEFORMAT (VERPFLICHTEND):
Gib deine Antwort NUR als gültiges JSON-Objekt zurück, ohne zusätzlichen Text.

Das JSON-Format muss exakt wie folgt aussehen:
{
  "is_outlier": true/false,
  "begruendung": "Kurze (1–2 Sätze) Erklärung, warum die Werte im Kontext als unauffällig (false) oder deutlich ungewöhnlich (true) eingestuft werden.",
  "zielspalten_bewertet": ["replies", "views", "votes", "price", "saving"]
}
"""
print("SYSTEM_PROMPT (Ausreißerbewertung mit erweitertem Fokus) definiert.")


# --- 3.2: Definition der USER_PROMPT Funktion ---
def erstelle_user_prompt(daten_zeile):
    """Konvertiert eine Pandas Series (Zeile) in einen formatierten JSON-String."""
    try:
        zeilen_dict = daten_zeile.to_dict()
        zeilen_json = json.dumps(zeilen_dict, indent=2, ensure_ascii=False)
        user_prompt = f"""
Hier sind die Daten für eine einzelne Zeile im JSON-Format. 
Bewerte diese Zeile gemäß den Anweisungen im SYSTEM_PROMPT.

DATENZEILE:
{zeilen_json}
"""
        return user_prompt
    except Exception:
        return None


# --- 3.3: HILFSFUNKTION: API-AUFRUF MIT BACKOFF ---
def rufe_claude_api_mit_backoff(system_prompt, user_prompt, model, max_retries=5):
    """Führt den Anthropic API-Aufruf mit Exponential Backoff durch."""
    for attempt in range(max_retries):
        try:
            response = client.messages.create(
                model=model,
                max_tokens=MAX_TOKENS,
                system=system_prompt,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=0.0
            )
            raw_json_text = response.content[0].text.strip()
            
            # Robuste JSON-Extraktion
            start = raw_json_text.find('{')
            end = raw_json_text.rfind('}') + 1
            
            if start != -1 and end != -1 and end > start:
                extracted_json = raw_json_text[start:end]
                return json.loads(extracted_json)
            else:
                raise json.JSONDecodeError("JSON nicht extrahierbar.", raw_json_text, 0)
            
        except RateLimitError:
            delay = 2 ** attempt
            time.sleep(delay)
            continue
        except Exception:
            return None
    return None


print("Hilfsfunktionen definiert.")
print("=" * 70)

#
################################################################################

# SCHRITT 4: HAUPTVERARBEITUNGSSCHLEIFE (EXECUTION)
print("\n--- Schritt 4: Ausführung der Ausreißererkennung auf dem gesamten Datensatz ---")

ergebnis_liste = []
fehler_indizes = []

print(f"Starte LLM-Analyse für {df_llm_input.shape[0]} Zeilen mit '{MODELL_NAME}'...")
print("-" * 70)

# Laufzeitmessung START (Experiment 3.5, LLM-Detektion auf LLM-Basis)
start_zeit = time.time()

for index, row in df_llm_input.iterrows():
    
    if index % 25 == 0 and index > 0:
        print(f"  ...verarbeite Zeile {index} von {df_llm_input.shape[0]}")
        time.sleep(1)  # Kleine Pause
        
    user_prompt = erstelle_user_prompt(row)
    
    ergebnis_dict = rufe_claude_api_mit_backoff(SYSTEM_PROMPT, user_prompt, MODELL_NAME)
    
    if ergebnis_dict:
        ergebnis_dict['original_index'] = row['original_index']
        ergebnis_liste.append(ergebnis_dict)
    else:
        fehler_indizes.append(index)
        print(f"  [FEHLER/FALLBACK] Zeile {index} übersprungen.")

# Laufzeitmessung ENDE
end_zeit = time.time()
laufzeit_sek = end_zeit - start_zeit
durchschnitt_pro_zeile = laufzeit_sek / df_llm_input.shape[0]

print("-" * 70)
print(f"LLM-Gesamtlaufzeit (Experiment 3.5, LLM-Basis): {laufzeit_sek:.2f} Sekunden "
      f"(≈ {laufzeit_sek/60:.2f} Minuten).")
print(f"Durchschnittliche Laufzeit pro Zeile: {durchschnitt_pro_zeile:.2f} Sekunden.")
print("-" * 70)
print("LLM-Verarbeitung abgeschlossen.")
#
################################################################################

# SCHRITT 5: Ergebnisse konsolidieren und speichern
print("\n--- Schritt 5: Ergebnisse konsolidieren und speichern ---")

if len(ergebnis_liste) == 0:
    print("FEHLER: Keine Ergebnisse zum Speichern vorhanden.")
    sys.exit("Skript gestoppt.")

df_ergebnisse_llm = pd.DataFrame(ergebnis_liste)
df_ergebnisse_llm = df_ergebnisse_llm.set_index('original_index').sort_index()

os.makedirs('ergebnisse', exist_ok=True)

# 1. Speichern der Indizes (quantitativer Vergleich)
try:
    df_nur_ausreisser = df_ergebnisse_llm[
        df_ergebnisse_llm['is_outlier'] == True
    ]
    df_nur_ausreisser.index.to_series().to_csv(
        DATEIPFAD_OUTPUT_INDIZES,
        index=False,
        header=['Ausreisser_Index']
    )
    anzahl_gefundener_ausreisser = len(df_nur_ausreisser)
    print(f"Quantitative Indizes gespeichert in: '{DATEIPFAD_OUTPUT_INDIZES}'")
except Exception as e:
    print(f"FEHLER beim Speichern der Indizes: {e}")
    anzahl_gefundener_ausreisser = 0


# 2. Speichern der kompletten Ergebnisse (qualitativer Vergleich)
try:
    df_ergebnisse_llm.to_csv(DATEIPFAD_OUTPUT_KOMPLETT, index=True)
    print(f"Komplette Ergebnisse (inkl. Begründungen) gespeichert in: '{DATEIPFAD_OUTPUT_KOMPLETT}'")
except Exception as e:
    print(f"FEHLER beim Speichern der kompletten Ergebnisse: {e}")


# --- SCHRITT 6: Zusammenfassung (Protokoll) ---
print("\n--- SCHRITT 6: Zusammenfassung EXPERIMENT 3.5 (LLM-Detektion - LLM-Basis) ---")
print("Methode:           Moderne KI: LLM (Claude 4.5 Sonnet)")
print(f"Basisdaten:        {DATEIPFAD_LLM_REPAIRED}")
print(f"Zielspalten:       {zielspalten} (5 Spalten)")
print(f"ERGEBNIS (COUNT):  {anzahl_gefundener_ausreisser} einzigartige Ausreißerzeilen identifiziert.")
print(f"Übersprungene Zeilen: {len(fehler_indizes)}")
print("=" * 70)


Lade Anthropic-Bibliothek...
Anthropic (Claude) Bibliothek erfolgreich geladen.
Alle Bibliotheken sind bereit.

--- Schritt 2: Reparierte Daten laden & API-Client initialisieren ---
Datensatz geladen: ergebnisse/2.2_rfd_repaired_llm.csv (1326 Zeilen)
STATUS: Spalten 'price'/'saving' für LLM-Analyse vorbereitet (Float/NaN-bereinigt).
Anthropic (Claude) API-Client und Modell 'claude-sonnet-4-5-20250929' initialisiert.

--- Schritt 3: Prompt-Strategie und Hilfsfunktionen ---
SYSTEM_PROMPT (Ausreißerbewertung mit erweitertem Fokus) definiert.
Hilfsfunktionen definiert.

--- Schritt 4: Ausführung der Ausreißererkennung auf dem gesamten Datensatz ---
Starte LLM-Analyse für 1326 Zeilen mit 'claude-sonnet-4-5-20250929'...
----------------------------------------------------------------------
  ...verarbeite Zeile 25 von 1326
  ...verarbeite Zeile 50 von 1326
  ...verarbeite Zeile 75 von 1326
  ...verarbeite Zeile 100 von 1326
  ...verarbeite Zeile 125 von 1326
  ...verarbeite Zeile 150 von 132